In [ ]:
# Vraag: de cutoff-time bepalen: tot op welk uur nemen we de requests elke dag mee in de analyse? Of: zijn er meerdere golven van requests en moeten we dan die meerdere golven analyseren? 
# Requests die buiten die ene of meerdere golven vallen, beschouwen we als niet-relevant. 
# In opdracht stond ook dat wat 's avonds nog loopt niet relevant is. 

# Deze analyse op basis van de index_files.py die ik aangemaakt heb op basis van de request en response files. 

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FixedLocator

from learning_driver_preferences.paths import OUTPUT, DATA_DIR

df = pd.read_csv(OUTPUT / "index_files.csv")

times = pd.to_datetime(df["request_time"], format="%H:%M:%S", errors="coerce")
minute_of_day = (times.dt.hour * 60 + times.dt.minute).dropna().astype(int)

# Count per minute and ensure all minutes 0..1439 exist
counts = minute_of_day.value_counts().sort_index()
counts = counts.reindex(range(1440), fill_value=0)

plt.figure(figsize=(14, 4))
plt.plot(counts.index, counts.values, color="#1a73e8", linewidth=1.2)

ax = plt.gca()
ax.set_xlim(0, 1440)

# Major ticks hourly with labels
hours = list(range(0, 1440, 60))
ax.xaxis.set_major_locator(FixedLocator(hours))
ax.set_xticklabels([f"{h:02d}:00" for h in range(24)])

# Minor ticks every 15 minutes
ax.xaxis.set_minor_locator(MultipleLocator(15))

# Style ticks
ax.tick_params(axis="x", which="major", length=7, width=1.0, color="#333")
ax.tick_params(axis="x", which="minor", length=4, width=0.8, color="#888", labelbottom=False)

# Helper grid lines 
ax.grid(which="minor", axis="x", linestyle=":", linewidth=0.6, color="#bbb", alpha=0.5)
ax.grid(which="major", axis="x", linestyle="--", linewidth=0.8, color="#888", alpha=0.7)

ax.set_xlabel("Time of the day")
ax.set_ylabel("Number of requests")
ax.set_title("Requests per minute")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FixedLocator

from learning_driver_preferences.paths import OUTPUT, DATA_DIR

CUTOFF_HOUR = 19  # alles vanaf 19:00 wordt uitgesloten

# --- Load ---
df = pd.read_csv(OUTPUT / "index_files.csv")

# --- Parse time & filter ---
times = pd.to_datetime(df["request_time"], format="%H:%M:%S", errors="coerce")
mask = times.dt.hour < CUTOFF_HOUR            # True = < 19:00
times_filtered = times[mask].dropna()

# --- Minute-of-day berekenen (alleen < 19:00) ---
minute_of_day = (times_filtered.dt.hour * 60 + times_filtered.dt.minute).astype(int)

# Count per minuut en zorg dat alle minuten 0..(19*60-1) bestaan
max_minute = CUTOFF_HOUR * 60 - 1             # 1139
counts = minute_of_day.value_counts().sort_index()
counts = counts.reindex(range(max_minute + 1), fill_value=0)

# --- Plot ---
plt.figure(figsize=(14, 4))
plt.plot(counts.index, counts.values, color="#1a73e8", linewidth=1.2)

ax = plt.gca()
ax.set_xlim(0, max_minute)

# Major ticks per uur met labels 00:00..19:00
hours = list(range(0, (CUTOFF_HOUR+1)*60, 60))   # 0..1140 stap 60
ax.xaxis.set_major_locator(FixedLocator(hours))
ax.set_xticklabels([f"{h:02d}:00" for h in range(0, CUTOFF_HOUR+1)])

# Minor ticks elke 15 min
ax.xaxis.set_minor_locator(MultipleLocator(15))

# Styling
ax.tick_params(axis="x", which="major", length=7, width=1.0, color="#333")
ax.tick_params(axis="x", which="minor", length=4, width=0.8, color="#888", labelbottom=False)
ax.grid(which="minor", axis="x", linestyle=":", linewidth=0.6, color="#bbb", alpha=0.5)
ax.grid(which="major", axis="x", linestyle="--", linewidth=0.8, color="#888", alpha=0.7)

ax.set_xlabel("Time of the day")
ax.set_ylabel("Number of requests")
ax.set_title("Requests per minute (filtered: before 19:00)")
plt.tight_layout()
plt.show()


In [ ]:
# Hier zelfde analyse, maar dan op basis van de Excel ModifiedQueryRows.xlsx
# Ik heb hier de analyse gemaakt op alle rijen in de Excel - excluding "shiftId"
# Eventueel filter op depot toepassen - bv. 0521; of op andere patronen. Eventueel subplots voor alle depots, maar wellicht zou dat te ver leiden. 

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

from learning_driver_preferences.paths import OUTPUT, DATA_DIR

# Read Excel
df = pd.read_excel(DATA_DIR / "ModifiedQueryRows.xlsx", engine="openpyxl")

# Normalize headers
df = df.rename(columns={c: c.strip() for c in df.columns})

# shiftId uitsluiten 
mask = df["RouteId"].astype(str).str.strip().str.lower() != "shiftid"
df = df.loc[mask].copy()

# Eventueel depotfilter toepassen, bv. 0521:
# df = df[df["RouteId"].astype(str).str.startswith("0521")]

# Parse the time column: STRICT format = "h:mm:ss.mmm AM/PM"
time_str = df["Time"].astype(str).str.strip()
times = pd.to_datetime(time_str, format="%I:%M:%S.%f %p", errors="coerce")

# Sanity check
parsed_ok = times.notna().sum()
print(f"Parsed {parsed_ok}/{len(times)} time values.")
if parsed_ok == 0:
    # Show a few samples to see what's going on
    print("Examples:", time_str.head(10).tolist())
    raise ValueError("No times parsed. Check the column name and sample values.")

# Convert to minute-of-day (0..1439)
minute_of_day = (times.dt.hour * 60 + times.dt.minute).dropna().astype(int)

# Count per minute and fill all 0..1439 minutes, so the x-axis is continuous
counts = minute_of_day.value_counts().sort_index().reindex(range(1440), fill_value=0)

# Plot (line) with hourly labels 
plt.figure(figsize=(12, 4))
plt.plot(counts.index, counts.values, color="#1a73e8", linewidth=1.2)

ax = plt.gca()
ax.set_xlim(0, 1440)

# Major ticks (one per hour, with labels 00:00..23:00)
hours = list(range(0, 1440, 60))
ax.set_xticks(hours)
ax.set_xticklabels([f"{h:02d}:00" for h in range(24)])

# Minor ticks only as markers (no labels).
ax.xaxis.set_minor_locator(MultipleLocator(15))
ax.tick_params(axis="x", which="major", length=7, width=1.0, color="#333")
ax.tick_params(axis="x", which="minor", length=4, width=0.8, color="#888", labelbottom=False)

ax.set_xlabel("Tijd van de dag")
ax.set_ylabel("Aantal requests")
ax.set_title("Requests per minuut (Excel)")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FixedLocator

from learning_driver_preferences.paths import DATA_DIR

CUTOFF_HOUR = 19  # alles vanaf 19:00 wordt uitgesloten

# --- Read Excel ---
df = pd.read_excel(DATA_DIR / "ModifiedQueryRows.xlsx", engine="openpyxl")

# Normalize headers
df = df.rename(columns={c: c.strip() for c in df.columns})

# shiftId uitsluiten
mask = df["RouteId"].astype(str).str.strip().str.lower() != "shiftid"
df = df.loc[mask].copy()

# (optioneel) depotfilter, bv. 0521:
# df = df[df["RouteId"].astype(str).str.startswith("0521")]

# --- Parse time (STRICT + fallback) ---
time_str = df["Time"].astype(str).str.strip()
times = pd.to_datetime(time_str, format="%I:%M:%S.%f %p", errors="coerce")
if times.notna().sum() == 0:
    # fallback als het formaat varieert
    times = pd.to_datetime(time_str, errors="coerce")

parsed_ok = int(times.notna().sum())
print(f"Parsed {parsed_ok}/{len(times)} time values.")
if parsed_ok == 0:
    raise ValueError("No times parsed. Check 'Time' format, e.g. '1:23:45.678 PM'.")

# --- Filter: alleen tijden vóór 19:00 ---
times_f = times[times.dt.hour < CUTOFF_HOUR].dropna()

# Minute-of-day (0..1439) maar alleen < 19:00
minute_of_day = (times_f.dt.hour * 60 + times_f.dt.minute).astype(int)

# Count per minuut; vul alle minuten 0..(19*60-1)
max_minute = CUTOFF_HOUR * 60 - 1  # 1139
counts = minute_of_day.value_counts().sort_index()
counts = counts.reindex(range(max_minute + 1), fill_value=0)

# Plot (line) with hourly labels 
plt.figure(figsize=(12, 4))
plt.plot(counts.index, counts.values, color="#1a73e8", linewidth=1.2)

ax = plt.gca()
ax.set_xlim(0, max_minute)

# Major ticks (one per hour, with labels 00:00..23:00)
hours = list(range(0, max_minute, 60))
ax.set_xticks(hours)
ax.set_xticklabels([f"{h:02d}:00" for h in range(CUTOFF_HOUR)])

# Minor ticks only as markers (no labels).
ax.xaxis.set_minor_locator(MultipleLocator(15))
ax.tick_params(axis="x", which="major", length=7, width=1.0, color="#333")
ax.tick_params(axis="x", which="minor", length=4, width=0.8, color="#888", labelbottom=False)

ax.set_xlabel("Tijd van de dag")
ax.set_ylabel("Aantal requests")
ax.set_title("Requests per minuut (Excel)")
plt.tight_layout()
plt.show()

